In this workshop we are goin to work with the dataset of the UPM researchers and their publications. This dataset is composed by two files: a json and a csv. These files are an extended versión of the csv that we used in the web application workshops.

Our goal is obtain as much information as we can from this dataset. We will achieve this in two ways: we will answer questions with numeric values (e.g. what is the average number publications of an UPM reaearcher? or, how many citations have the reseachers, suming all their publications?) or we will generate some charts to present information in a visual way (e.g. how is the relation between the publication year of a paper and its number of citations? or, how is the histogram of the number of publications by researcher?)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy
#Faltan S1 y S12

# SUJETO 2

In [ ]:
datasetS2 = pd.read_pickle("/media/jaime/tocho/Universidad/Master/TFM/WESAD/S2/S2.pkl")

In [ ]:
emgS2 = datasetS2["signal"]["chest"]["EMG"]
emgDatasetS2=pd.DataFrame(emgS2,columns=["EMG"])
ecgS2 = datasetS2["signal"]["chest"]["ECG"]
ecgDatasetS2=pd.DataFrame(ecgS2,columns=["ECG"])
tempS2 = datasetS2["signal"]["chest"]["Temp"]
tempDatasetS2=pd.DataFrame(tempS2,columns=["TEMP"])
edaS2 = datasetS2["signal"]["chest"]["EDA"]
edaDatasetS2=pd.DataFrame(edaS2,columns=["EDA"])
respS2 = datasetS2["signal"]["chest"]["Resp"]
respDatasetS2=pd.DataFrame(respS2,columns=["RESP"])
labelS2 = datasetS2["label"]
labelDatasetS2=pd.DataFrame(labelS2,columns=["label"])
dataChestS2=pd.concat([ecgDatasetS2, emgDatasetS2, edaDatasetS2, tempDatasetS2, respDatasetS2, labelDatasetS2], axis=1)
dataChestS2

In [ ]:
dataChestS2.groupby(dataChestS2['label'])["ECG"].count().plot.bar()
#Delete 0,5,6,7

In [ ]:
dataChestCleanS2 = dataChestS2.loc[dataChestS2["label"]>0]
dataChestCleanS2 = dataChestCleanS2.loc[dataChestCleanS2["label"]<5]
dataChestCleanS2
# 1 -> baseline
# 2 -> stress
# 3 -> amusement
# 4 -> meditation

In [ ]:
dataChestCleanS2.groupby(dataChestS2['label'])["ECG"].count().plot.bar()

# SUJETO 3

In [ ]:
datasetS3 = pd.read_pickle("/media/jaime/tocho/Universidad/Master/TFM/WESAD/S3/S3.pkl")

In [ ]:
emgS3 = datasetS3["signal"]["chest"]["EMG"]
emgDatasetS3=pd.DataFrame(emgS3,columns=["EMG"])
ecgS3 = datasetS3["signal"]["chest"]["ECG"]
ecgDatasetS3=pd.DataFrame(ecgS3,columns=["ECG"])
tempS3 = datasetS3["signal"]["chest"]["Temp"]
tempDatasetS3=pd.DataFrame(tempS3,columns=["TEMP"])
edaS3 = datasetS3["signal"]["chest"]["EDA"]
edaDatasetS3=pd.DataFrame(edaS3,columns=["EDA"])
respS3 = datasetS3["signal"]["chest"]["Resp"]
respDatasetS3=pd.DataFrame(respS3,columns=["RESP"])
labelS3 = datasetS3["label"]
labelDatasetS3=pd.DataFrame(labelS3,columns=["label"])
dataChestS3=pd.concat([ecgDatasetS3, emgDatasetS3, edaDatasetS3, tempDatasetS3, respDatasetS3, labelDatasetS3], axis=1)
dataChestS3

In [ ]:
dataChestCleanS3 = dataChestS3.loc[dataChestS3["label"]>0]
dataChestCleanS3 = dataChestCleanS3.loc[dataChestCleanS3["label"]<5]
dataChestCleanS3

# SUJETO 4

# Dataset exploration
Any project related with data analysis starts with a study of the data itself: 
* What kind of data do we have?
* How is it organized? How many columns have each dataframe?
* Are there wrong or missing values?
* What does each value of a column means? How many different values are in each column?

With the examples we saw in the previous workshop try to answer that questions.

# Data cleaning
After a first view of the data and before to start working with it, we must clean it. This proccess is called preprocessing and it is crucial to be able to obtain good results. We will discard invalid data, fill missing values, drop redundant information, etc. We need to create a dataset with the following restrictions.
* All reserachers should have name, last_name and auth_id. All publications should have title, journal and document_id. Drop the rows that do not match this condition (check https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)
* There are no duplicated publications (same title) or researchers (same name and lastname). To find duplicates check https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html
* All numeric fields must be an integer number bigger or equal to 0. Replace missing values with 0, round decimal values

In [ ]:
researchersClean = researchers.dropna(0,'any',subset=["name","last_name","auth_id"])
publicationsClean = publications.dropna(0,'any',subset=["title","journal","document_id"])
researchersClean2 = researchersClean.drop_duplicates(["name","last_name"])
publicationsClean2 = publicationsClean.drop_duplicates("title")
#researchersClean2.loc[researchersClean2["publications"]='NaN']
#researchersClean2.loc[researchersClean2["citations"]<0]
#researchersClean2.loc[researchersClean2["cited_by"]<0]
#publicationsClean2.loc[publicationsClean2["cited_count"]<0]
publicationsClean3 = publicationsClean2.fillna({'cited_count': 0})
researchersClean3 = researchersClean2.fillna({'citations': 0,'cited_by': 0,'publications': 0})

Finally, to easy some calculations, lets convert the date column of the publications dataset to a datetime type

In [ ]:
publicationsClean3.date = pd.to_datetime(publications.date, errors='coerce')

# Data analytics

Once we know about the data we are using and we have cleaned it, we need to ask what kind of useful information we can extract from that data. It is a good idea to make a kind of brainstorming of possible questions, then take the list of resulting questions and sort them by their difficulty, finally begin to answer them starting by the easiest ones.

Yoy may need to use a joined dataset to answer some questions

In [ ]:
# This operation creates a dataset combining researchers and publications producing a dataset with one row for each
# combination of researcher-publication in which the researcher is one of the authors of the publication
data = researchersClean3.merge(publicationsClean3.rename(columns={"authors":"auth_id"}).explode("auth_id"), on="auth_id")

## Basic results

Some of the esaiest question or more basic results we can obtain are the following:
* Numeric results
    * How many researchers are?
    * How many publications are?
    * How many of journals are?
    * How many types of publications are?
    * How many publications of each type are?
    * How many publications have each researcher in average? (Use only researchers dataset)
    * Which is the oldest publication?
    * Which is the publication with most citations?
    * Who is the researcher with more publications? (Use only researchers dataset)
* Graphic results
    * Histogram of the number of publications per researcher
    * Bar chart of the number of publications by type
    * Line chart of the number of publications per year

Som of those questions are just countings, sumations or rankings, but other need data aggregations with ``groupby``.

**Answer each question in a different cell of this notebook (remember that you can insert new cell from the Insert menu).**

In [ ]:
data

In [ ]:
data["auth_id"].unique().size #unique

In [ ]:
data["title"].unique().size #unique

In [ ]:
data["journal"].unique().size 

In [ ]:
data["type"].describe()[1]

In [ ]:
data.groupby(data['type'])['type'].count()

In [ ]:
researchersClean3["publications"].mean()

In [ ]:
data[data["date"]==data["date"].min()] #pub mas antigua

In [ ]:
data[data["cited_count"]==data["cited_count"].max()]

In [ ]:
researchersClean3[researchersClean3["publications"]==researchersClean3["publications"].max()]

In [ ]:
data["publications"]

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
ax.hist(researchers["publications"])
plt.show(fig)

In [ ]:
publications.groupby(publications['type'])["title"].count().plot.bar()


In [ ]:
publicationsClean3.groupby(publicationsClean3['date'].dt.year)["date"].count().plot.line()

# Intermediate results

* Numeric results
    * How many publications of each researcher are in the data? Is this the same number than the column ``document-count``?
    * How many authors have each publication? (show 5)
    * What is the average number of authors in a publication?
    * What is the average number of pages in a publication depending of the publication type?
    * What is the average number of citations in a publication depending of the publication type?
    * How many years each researcher have been active? (difference between the publciationDate of the oldest and newest publications)
* Graphic results
    * Histogram of the number of researchers by the number of active years
    * Histogram of the number of authors in a publication
    * Scatter plot of the publication year vs the number of citations
    * Chart of the average number of citations depending on the publication type
    * Chart of the average number of pages depending on the publication type

**Choose 5 questions from the list and answer each in a different cell.**

In [ ]:
data.groupby(data['auth_id'])['auth_id'].count()  #Nºpubs/researcher
#data[data["auth_id"]==6503890565] No Coincide

In [ ]:
data.groupby(data['auth_id']).first()[data.groupby(data['auth_id'])['auth_id'].count() != data.groupby(data['auth_id'])['publications'].first()]

In [ ]:
def ret_five(x):
    if x is None or len(x)<5:
        if x:
            print(len(x),x)
            #return
        #return 
    else:
        print(len(x),[x[0],x[1],x[2],x[3],x[4]])
        #return 
publicationsClean3['authors'].apply(ret_five)


In [ ]:
def ret_count(x):
    if x is None:
        return 0
    else:
        return len(x)
publicationsClean3.authors.apply(ret_count).mean()

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
ax.hist(publicationsClean3.authors.apply(ret_count))
plt.show(fig)

In [ ]:
publicationsClean3.groupby('type').cited_count.mean()

## Advanced results

The most difficult questions to answer usually are a small set which needs more explanation than a single line.

* Coauthor: 
    One researcher is coatuhor with other if both appears as author in at least one publication. 
    * What is the average number of different coauthors of each researcher?
    * How is the most frequent coauthor (their id) of each researcher?


* H-index: 
    The h-index (https://en.wikipedia.org/wiki/H-index) is a metric that tries to describe the impact of a researcher based on the number of citations received in each publication. A researcher has a h-index of `n` if she has at least `n` publications with `n` or more citations.

    * What is the h-index of each researcher?
    * Chart of the publications and number of citations of a certain researcher, highlinting (in a different color) which publications contributed to the h-index

![alt text](img/h-index.svg "H-Index")

**Choose 1 group and solve it.**

In [ ]:
index_H = {}
for authID, group in data.groupby("auth_id"):
    #print(group["cited_count"])
    i = 1
    while True:
        cnt = 0
    #print("HOAL",authID,group["cited_count"])
        for cite in group["cited_count"]:
            if cite >= i:
                cnt+=1
        if cnt >= i:
            i+=1
        else:
            index_H[authID]=i-1
            break


In [ ]:
#6504374283: 8
authID = 6504374283
gteHIndex = []
gteIndex = []
ltHIndex = []
ltIndex = []
def getPubHIndex(x,HIndex):
    if x >= HIndex:
        gteHIndex.append(x)
    else:
        ltHIndex.append(x)
data[data["auth_id"]==authID]["cited_count"].apply(getPubHIndex, HIndex=index_H[authID])
gteHIndex.sort(reverse=True)
ltHIndex.sort(reverse=True)
for i in range(len(gteHIndex)):
    gteIndex.append(i+1)
for j in range(len(ltHIndex)):
    ltIndex.append(j+1+len(gteHIndex))
fig, ax = plt.subplots(figsize=(7,5))
ax.bar(gteIndex,gteHIndex,color='g')
ax.bar(ltIndex,ltHIndex,color='r')
ax.set_xlabel("Publications")
ax.set_ylabel("Cites")
ax.set_title("H-Index = " + str(index_H[authID]))
plt.show(fig)